In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
DATADIR = "positiondata"

In [ ]:
fnames = os.listdir(DATADIR)
case1_cams = [os.path.join(DATADIR, i) for i in fnames if "_1.npy" in i]
case2_cams = [os.path.join(DATADIR, i) for i in fnames if "_2.npy" in i]
case3_cams = [os.path.join(DATADIR, i) for i in fnames if "_3.npy" in i]
case4_cams = [os.path.join(DATADIR, i) for i in fnames if "_4.npy" in i]

In [ ]:
def read_case(cams):
    cam_names = [i.split("_")[0][-1] for i in cams]
    data = [np.load(i) for i in cams]
    return list(zip(cam_names, data))

In [ ]:
def plot_case(case, title):
    assert(len(case) == 3)
    fig, axes = plt.subplots(3, 2, sharex=True, sharey=True)
    fig.set_size_inches(8, 10)
    fig.suptitle(title)
    for i, cam in enumerate(case):
        axes[i, 0].title.set_text("Camera {}, x".format(cam[0]))
        axes[i, 0].set_xlabel("Sample")
        axes[i, 0].set_ylabel("Pixel location")
        axes[i, 0].plot(cam[1][0])
        axes[i, 1].title.set_text("Camera {}, y".format(cam[0]))
        axes[i, 1].set_xlabel("Sample")
        axes[i, 1].set_ylabel("Pixel location")
        axes[i, 1].plot(cam[1][1])

In [ ]:
case_1 = read_case(case1_cams)
case_2 = read_case(case2_cams)
case_3 = read_case(case3_cams)
case_4 = read_case(case4_cams)
plot_case(case_1, "Case 1")
plot_case(case_2, "Case 2")
plot_case(case_3, "Case 3")
plot_case(case_4, "Case 4")

In [ ]:
#x = np.stack((case_1[0][1], case_1[1][1], case_1[2][1]))
def shortest_length(case):
    return min([i[1].shape[1] for i in case])

def stack_observations(case):
    last = shortest_length(case)
    arrays = []
    for cam in case:
        x = cam[1][0][:last]
        y = cam[1][1][:last]
        arrays.append(x)
        arrays.append(y)
    return np.stack(tuple(arrays))

In [ ]:
stack_observations(case_1).shape